In [30]:
import numpy as np
from itertools import zip_longest
# Load data
inp_txt = open("input.txt").read()

## Part "Demo"

Create dictionary for range 0..8. zip_longest merges two lists and takes the longer as leading list
Since second list `[]` is an empty array, we set `fillvalue=0` to set `0` as value for every key in `range(9)`.

In [31]:
i1 = dict(zip_longest(range(9), [], fillvalue=0))
print(f"{i1=}")

i1={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}


Split input line to array of ints

In [32]:
arr_int = [int(x) for x in inp_txt.split(",")]
print(f"{arr_int=}")

arr_int=[5, 3, 2, 2, 1, 1, 4, 1, 5, 5, 1, 3, 1, 5, 1, 2, 1, 4, 1, 2, 1, 2, 1, 4, 2, 4, 1, 5, 1, 3, 5, 4, 3, 3, 1, 4, 1, 3, 4, 4, 1, 5, 4, 3, 3, 2, 5, 1, 1, 3, 1, 4, 3, 2, 2, 3, 1, 3, 1, 3, 1, 5, 3, 5, 1, 3, 1, 4, 2, 1, 4, 1, 5, 5, 5, 2, 4, 2, 1, 4, 1, 3, 5, 5, 1, 4, 1, 1, 4, 2, 2, 1, 3, 1, 1, 1, 1, 3, 4, 1, 4, 1, 1, 1, 4, 4, 4, 1, 3, 1, 3, 4, 1, 4, 1, 2, 2, 2, 5, 4, 1, 3, 1, 2, 1, 4, 1, 4, 5, 2, 4, 5, 4, 1, 2, 1, 4, 2, 2, 2, 1, 3, 5, 2, 5, 1, 1, 4, 5, 4, 3, 2, 4, 1, 5, 2, 2, 5, 1, 4, 1, 5, 1, 3, 5, 1, 2, 1, 1, 1, 5, 4, 4, 5, 1, 1, 1, 4, 1, 3, 3, 5, 5, 1, 5, 2, 1, 1, 3, 1, 1, 3, 2, 3, 4, 4, 1, 5, 5, 3, 2, 1, 1, 1, 4, 3, 1, 3, 3, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 5, 1, 2, 2, 5, 2, 4, 1, 1, 2, 4, 1, 2, 3, 4, 1, 2, 1, 2, 4, 2, 1, 1, 5, 3, 1, 4, 4, 4, 1, 5, 2, 3, 4, 4, 1, 5, 1, 2, 2, 4, 1, 1, 2, 1, 1, 1, 1, 5, 1, 3, 3, 1, 1, 1, 1, 4, 1, 2, 2, 5, 1, 2, 1, 3, 4, 1, 3, 4, 3, 3, 1, 1, 5, 5, 5, 2, 4, 3, 1, 4]


Run "Grouby" to count the fishs for every day

In [33]:
gp = np.unique(arr_int, return_counts=True)
print(f"{gp=}")

gp=(array([1, 2, 3, 4, 5]), array([109,  52,  43,  54,  42], dtype=int64))


Build dict from gp. Attention: usually zip takes two arrays of same length and merges them like a zipper. Since gp is a list containing two lists, we have to unpack it. This is done by using `*` to unpack `gp`.

In [34]:
i2 = dict(zip(*gp))
print(f"{i2=}") 

i2={1: 109, 2: 52, 3: 43, 4: 54, 5: 42}


Now we have to dicts. One (`inp1`) containing all days/indexes, but with all values `0`. And one (`inp2`) containing all the start populations for each day, but not for all days. By help of the `|`-operator we can easily merge them.

In [35]:
inp =  i1 | i2
print(f"{inp=}")

inp={0: 0, 1: 109, 2: 52, 3: 43, 4: 54, 5: 42, 6: 0, 7: 0, 8: 0}


Now we only have to call the `evolve`-function and pass the input and the cycles/evolutions to be run...

In [36]:
def evolve(nums, cycles: int) -> int:
    print(f"{cycles=}, population={nums}")
    # In each cycle we have to lower the day count until the next birth.
    # We could either move the fish-count from nums[1] to nums[0] (and do it for every other position) 
    # or (and thats is what we do) we instead move the indexes.
    print(f"Current keys: {nums.keys()}")

    last_key = list(nums.keys())[-1:] #Pick from index -1 (last el-1)
    all_but_last_key = list(nums.keys())[:-1] #Pick up to index -1 (last el-1)
    rotated_keys = last_key+all_but_last_key #Join them together
    print(f"Rotated keys:           {rotated_keys}")

    # By roting every fish that was in day 3 is now in day 2, day 2 becomes day 1, ...
    # and day 0 becomes to(?) day 8(!). So we automatically spawn a new baby fish at day 8 for each fish in day 0
    # Hm, but what about the birthgiving fish? Each mother fish from day 0 should also move to day 6!
    # Thats why we also have to update the values and not only the keys. Thus we simply add the amount of fish from day 0
    # to day 6. Problem solved! (We add them instead of setting them, because there could be already fish from day 7...)

    # With update we update the fish dict with a new value (nums[7]+nums[0]) at index 7. (We use 7 instead of six, because we haven't rotated yet.)
    # Since update is in-place and doesn't return any value we use "or" to get a value, if left-hand is None. TL;DR: Update takes place,
    # returns "None" thus "or" kicks in and returns value (which are now updated)
    print(f"Current values: {nums.values()}")
    new_values = nums.update({7: nums[7]+nums[0]}) or nums.values()
    print(f"Manipula. vals: {new_values}")
    # No we create a new dict with the rotated keys and the patched values
    nums = dict(zip(rotated_keys, new_values))
    print(f"Manipulated nums/dict: {nums}\n===========================")

    # Note: The below is a short hand for if-else
    # If we are in the last cycle (cycles==1) then we build the sum, for each value of our dict. Since the values reflect the fish count per day we get the total fish count
    # If we aren't in the last cycle, we call evolve with cycles-1 and our current fish state dict.
    return sum([v for i, v in nums.items()]) if cycles == 1 else evolve(nums, cycles - 1)

print(f"After 80 days: {evolve(inp, 3)}")

cycles=3, population={0: 0, 1: 109, 2: 52, 3: 43, 4: 54, 5: 42, 6: 0, 7: 0, 8: 0}
Current keys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8])
Rotated keys:           [8, 0, 1, 2, 3, 4, 5, 6, 7]
Current values: dict_values([0, 109, 52, 43, 54, 42, 0, 0, 0])
Manipula. vals: dict_values([0, 109, 52, 43, 54, 42, 0, 0, 0])
Manipulated nums/dict: {8: 0, 0: 109, 1: 52, 2: 43, 3: 54, 4: 42, 5: 0, 6: 0, 7: 0}
cycles=2, population={8: 0, 0: 109, 1: 52, 2: 43, 3: 54, 4: 42, 5: 0, 6: 0, 7: 0}
Current keys: dict_keys([8, 0, 1, 2, 3, 4, 5, 6, 7])
Rotated keys:           [7, 8, 0, 1, 2, 3, 4, 5, 6]
Current values: dict_values([0, 109, 52, 43, 54, 42, 0, 0, 0])
Manipula. vals: dict_values([0, 109, 52, 43, 54, 42, 0, 0, 109])
Manipulated nums/dict: {7: 0, 8: 109, 0: 52, 1: 43, 2: 54, 3: 42, 4: 0, 5: 0, 6: 109}
cycles=1, population={7: 0, 8: 109, 0: 52, 1: 43, 2: 54, 3: 42, 4: 0, 5: 0, 6: 109}
Current keys: dict_keys([7, 8, 0, 1, 2, 3, 4, 5, 6])
Rotated keys:           [6, 7, 8, 0, 1, 2, 3, 4, 5]
Current value